# Working with a database

This week's assignment has a few basic steps.  First, we're going to pull some data down off the internet and store it into our MySQL database.  Make sure that you use your username as part of the table name as show in the examples so that you don't create a problem for other students.

Then, we'll merge that with some data already in the database and calculate a few results.  When it comes to calculating the results, you can do so either with SQL or with Pandas operations.


## PART 1: Setup your database connection and table name

In the code below, change the value of the variable `MYTABLE` to use your own username rather that `'pboal'`

You can then use `MYTABLE` in the rest of your code to reference that table name.

In [5]:
import getpass
from sqlalchemy import create_engine
import pandas as pd 


myname = getpass.getuser().split('-')[1]
myname
MYTABLE=myname + '_data'

host = 'slucor2022-instance-1.cgdcoitnku0k.us-east-1.rds.amazonaws.com'
port = "3306"
username = 'slucor2022'
password = "SLUcor2022"
database = "hds5210"

conn = create_engine("mysql+pymysql://" + username + ":" + password + "@" + host + "/" + database, echo = False)

In [6]:
assert(MYTABLE != 'pboal')
assert(conn.connect())

## PART 2: Bring in outside data

Grab the data from this URL and put it into a database table named with your `username_data`.

https://opendata.arcgis.com/api/v3/datasets/ed0bc689bde246b18835c7529ba4efb4_0/downloads/data?format=csv&spatialRefId=4326

By the end of your cell, the table should be created.  The tests are going to verify that the table exists and looks right.

In [4]:
df = pd.read_csv("https://opendata.arcgis.com/api/v3/datasets/ed0bc689bde246b18835c7529ba4efb4_0/downloads/data?format=csv&spatialRefId=4326")
df.to_sql(MYTABLE, conn)

ValueError: Table 'vaishnavig1411_data' already exists.

In [17]:
dxyz = pd.read_sql_query('SELECT * FROM ' + MYTABLE, conn)
assert(dxyz.shape == (323,22))
assert(list(dxyz.columns) == ['index', 'X', 'Y', 'OBJECTID', 'Provider_Name', 'NPI', 'CCN',
       'Business_Street_Address', 'Business_City', 'Business_County',
       'Business_ZIP_Code', 'Business_State_Territory', 'Payment_Year_Number',
       'Program_Type', 'Medicaid_EP_Hospital_Type', 'total_payments',
       'Last_Payment_Criteria', 'Recent_Disbursement_Amount', 'Latitude',
       'Longitude', 'Last_Program_Year', 'Last_Payment_Year'])

## PART 3: Combine with other data in the database

In the database, there is an existing table called `population`.  We want to merge the DHCS datafile loaded above with the population data available in this other database table  The tables can be merged on `MYTABLE`'s `Business_ZIP_Code` field and `population`'s `Zip` field.

Note that not all `Zip_Codes` from your downloaded file have to be in the `population` table.  If they aren't, then I want you to eliminate the non-matching records.  That is, only keep the records that have a matching ZIP code in both sets of data.

Answer the question:
Which providers are located in the zipcode with the largest population?

Put your answer in the form `answer = ['a', 'list', 'of', 'NPI', 'like', '1593042103]`

In [43]:
query = "SELECT * FROM {} mt LEFT JOIN population p ON mt.Business_ZIP_Code = p.Zip ".format(MYTABLE)
zip_merged = pd.read_sql_query(query, conn)
answer = zip_merged.loc[zip_merged['Population'] == zip_merged['Population'].max()]

answer = [str(npi) for npi in answer['NPI']]

['1194016923']


In [44]:
assert(type(answer) == list)
assert(answer == ['1194016923'])

## PART 4: Total by hospital type

This is a multistep process:
* From our downloaded data file, compute the `Total payments` per ZIP code and Medicaid EP Hospital Type.
* Then merge that with the `population` data to compute a `Total payments` per person.
* Then average that across all of the `Medicaid EP Hospital Types` to get an average per persona payment for these type of hospital.

Your answer should be in structure of a data frame with at least two columns:
* Medicaid_EP_Hospital_Type
* Avg_Pay_per_Capita

## First using SQL

In [55]:
query = """select Medicaid_EP_Hospital_Type, avg(population_per_zip_code) as Avg_Pay_per_Capita from
        (SELECT Medicaid_EP_Hospital_Type,
            SUM(total_payments)/Population as population_per_zip_code
           FROM {mytable} mt LEFT JOIN population p ON p.ZIP = mt.Business_ZIP_code 
                    GROUP BY mt.Business_ZIP_code, mt.Medicaid_EP_Hospital_Type)as sub group by Medicaid_EP_Hospital_Type
        """.format(mytable=MYTABLE)

answer = pd.read_sql_query(query, conn)
print(answer.head())

  Medicaid_EP_Hospital_Type  Avg_Pay_per_Capita
0      Acute Care Hospitals          106.582669
1      Children's Hospitals          329.550946


In [58]:
assert(type(answer) == pd.core.frame.DataFrame)
assert(round(answer.query("Medicaid_EP_Hospital_Type == 'Acute Care Hospitals'")['Avg_Pay_per_Capita'].sum(),3) == 106.583)
assert(round(answer.query("Medicaid_EP_Hospital_Type == 'Children\\'s Hospitals'")['Avg_Pay_per_Capita'].sum(),3) == 329.551)


In [59]:
answer

,Medicaid_EP_Hospital_Type,Avg_Pay_per_Capita
0,Acute Care Hospitals,106.582669
1,Children's Hospitals,329.550946


## Now using Pandas instead of SQL

In [24]:
mytable_df = pd.read_sql_query("SELECT * FROM {}".format(MYTABLE), conn)
population_df = pd.read_sql_query("SELECT * FROM population", conn)

In [49]:
# merged_df = pd.merge(mytable_df, population_df, left_on='Business_ZIP_Code', right_on='Zip', how='left')

total_payments = mytable_df.groupby(['Business_ZIP_Code', 'Medicaid_EP_Hospital_Type'])['total_payments'].sum().to_frame('total_payments').reset_index()
tp_merged_df = pd.merge(total_payments, population_df, left_on='Business_ZIP_Code', right_on='Zip', how='left')
tp_merged_df['tp_per_population'] = tp_merged_df['total_payments']/tp_merged_df['Population']
tp_merged_df.head()

,Business_ZIP_Code,Medicaid_EP_Hospital_Type,total_payments,index,Zip,Population,tp_per_population
0,90004,Acute Care Hospitals,1093021.73,115.0,90004.0,61105.0,17.887599
1,90012,Acute Care Hospitals,3614739.93,433.0,90012.0,37268.0,96.993129
2,90015,Acute Care Hospitals,10045484.04,738.0,90015.0,23900.0,420.313140
3,90017,Acute Care Hospitals,3899845.21,648.0,90017.0,27832.0,140.120912
4,90020,Children's Hospitals,427640.22,390.0,90020.0,39366.0,10.863187


In [51]:
avg_pay_per_capita_ht = tp_merged_df.groupby(['Medicaid_EP_Hospital_Type'])['tp_per_population'].mean()
avg_pay_per_capita_ht.head()

Medicaid_EP_Hospital_Type
Acute Care Hospitals    106.582669
Children's Hospitals    329.550946
Name: tp_per_population, dtype: float64



---



## Submitting Your Work

In order to submit your work, you'll need to use the `git` command line program to **add** your homework file (this file) to your local repository, **commit** your changes to your local repository, and then **push** those changes up to github.com.  From there, I'll be able to **pull** the changes down and do my grading.  I'll provide some feedback, **commit** and **push** my comments back to you.  Next week, I'll show you how to **pull** down my comments.

To run through everything one last time and submit your work:
1. Use the `Kernel` -> `Restart Kernel and Run All Cells` menu option to run everything from top to bottom and stop here.
2. Follow the instruction on the prompt below to either ssave and submit your work, or continue working.

If anything fails along the way with this submission part of the process, let me know.  I'll help you troubleshoort.

---

In [1]:
a=input('''
Are you ready to submit your work?
1. Click the Save icon (or do Ctrl-S / Cmd-S)
2. Type "yes" or "no" below
3. Press Enter

''')

if a=='yes':
    !git pull
    !git add week15_assignment.ipynb
    !git commit -a -m "Submitting the week 15 programming assignment"
    !git push
else:
    print('''
    
OK. We can wait.
''')


Are you ready to submit your work?
1. Click the Save icon (or do Ctrl-S / Cmd-S)
2. Type "yes" or "no" below
3. Press Enter

 



    
OK. We can wait.

